In [1]:
import pandas as pd
import sqlite3 as sql
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn import utils


In [2]:
stats_cnx = sql.connect('../../Data/Combined Clean Historical Data/Historical_Fantasy_Stats.db')
stats = pd.read_sql_query('SELECT * FROM Historical_Fantasy_Points ', stats_cnx)
stats_cnx.close()
stats['season'] = pd.to_datetime(stats['season'], format='%Y')
stats_prep = stats.copy()

# Manipulating Data

In [3]:
stats_prep['season'] = pd.to_datetime(stats_prep['season'], format='%Y')
stats_prep['nfanpts'] = stats_prep.groupby(['Name','player_id']).shift(-1)['fantast_pts']
stats_prep = stats_prep[stats_prep['nfanpts'].notna()]

# This is to take the players who record more then 40 fantasy points in a season.
stats_prep = stats_prep[stats_prep['nfanpts'] >= 40]
stats_prep

,Name,Position,player_id,season,fantast_pts,completions,attempts,passing_yards,passing_tds,interceptions,...,yac_sh,wopr,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,nfanpts
3,Donald Driver,WR,4541,2010-01-01,129.50,0,0,0,0,0,...,0.093617,0.391267,0.138957,0.142857,0.153439,0.152074,0.140907,0.139737,1.116601,117.50
6,Tony Gonzalez,TE,6101,2010-01-01,171.60,0,0,0,0,0,...,0.139824,0.421116,0.176107,0.214286,0.195000,0.197368,0.195197,0.183743,1.136915,209.50
7,Tony Gonzalez,TE,6101,2011-01-01,209.50,0,0,0,0,0,...,0.136058,0.448635,0.200458,0.241379,0.246512,0.245902,0.220919,0.208642,1.473064,234.00
8,Tony Gonzalez,TE,6101,2012-01-01,234.00,0,0,0,0,0,...,0.122271,0.456716,0.197076,0.250000,0.264228,0.262590,0.223538,0.207661,1.512195,218.90
10,Matt Hasselbeck,QB,7091,2010-01-01,148.04,266,444,3001,12,17,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,190.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5371,Josh Jacobs,RB,35700,2019-01-01,191.60,0,0,0,0,0,...,0.110507,0.088289,0.052432,0.000000,0.051282,0.045977,0.026216,0.041946,0.401937,231.30
5373,Miles Boykin,WR,35703,2019-01-01,50.80,0,0,0,0,0,...,0.010118,0.196293,0.073442,0.103448,0.072993,0.078313,0.088445,0.079443,0.578947,69.60
5375,Drew Lock,QB,35704,2019-01-01,68.00,100,156,1020,7,3,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,179.32
5377,Daniel Jones,QB,35710,2019-01-01,210.98,284,459,3027,24,12,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,178.02


In [4]:
stats_prep.rename(columns = {'nfanpts':'target'}, inplace = True)

In [5]:
unwanted_col = ['Name', 'Position', 'player_id','fantast_pts','rushing_epa', 
           'receiving_2pt_conversions', 'special_teams_tds', 'rushing_2pt_conversions',
           'rushing_fumbles_lost']

stats_prep.drop(unwanted_col, axis=1, inplace=True)
predictors = list(stats_prep)
unwanted_predictor = {'season', 'target'}
predictors = [e for e in predictors if e not in unwanted_predictor]

In [6]:
stats_prep

,season,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,...,yac_sh,wopr,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,target
3,2010-01-01,0,0,0,0,0,0,0,0,0,...,0.093617,0.391267,0.138957,0.142857,0.153439,0.152074,0.140907,0.139737,1.116601,117.50
6,2010-01-01,0,0,0,0,0,0,0,0,0,...,0.139824,0.421116,0.176107,0.214286,0.195000,0.197368,0.195197,0.183743,1.136915,209.50
7,2011-01-01,0,0,0,0,0,0,0,0,0,...,0.136058,0.448635,0.200458,0.241379,0.246512,0.245902,0.220919,0.208642,1.473064,234.00
8,2012-01-01,0,0,0,0,0,0,0,0,0,...,0.122271,0.456716,0.197076,0.250000,0.264228,0.262590,0.223538,0.207661,1.512195,218.90
10,2010-01-01,266,444,3001,12,17,29,175,6,5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,190.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5371,2019-01-01,0,0,0,0,0,0,0,0,0,...,0.110507,0.088289,0.052432,0.000000,0.051282,0.045977,0.026216,0.041946,0.401937,231.30
5373,2019-01-01,0,0,0,0,0,0,0,0,0,...,0.010118,0.196293,0.073442,0.103448,0.072993,0.078313,0.088445,0.079443,0.578947,69.60
5375,2019-01-01,100,156,1020,7,3,5,26,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,179.32
5377,2019-01-01,284,459,3027,24,12,38,295,12,8,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,178.02


In [7]:
stats_prep = stats_prep.set_index(['season'])
stats_prep = stats_prep.sort_index()

In [8]:
stats_prep.apply(pd.isnull).sum()

completions                    0
attempts                       0
passing_yards                  0
passing_tds                    0
interceptions                  0
sacks                          0
sack_yards                     0
sack_fumbles                   0
sack_fumbles_lost              0
passing_air_yards              0
passing_yards_after_catch      0
passing_first_downs            0
passing_epa                    0
passing_2pt_conversions        0
carries                        0
rushing_yards                  0
rushing_tds                    0
rushing_fumbles                0
rushing_first_downs            0
receptions                     0
targets                        0
receiving_yards                0
receiving_tds                  0
receiving_fumbles              0
receiving_fumbles_lost         0
receiving_air_yards            0
receiving_yards_after_catch    0
receiving_first_downs          0
receiving_epa                  0
games                          0
tgt_sh    

In [9]:
stats_prep['rtd_sh'] = stats_prep['rtd_sh'].fillna(stats_prep['rtd_sh'].mean())
stats_prep['dom'] = stats_prep['dom'].fillna(stats_prep['dom'].mean())
stats_prep['w8dom'] = stats_prep['w8dom'].fillna(stats_prep['w8dom'].mean())

In [10]:
X = stats_prep.drop("target", axis=1)
Y = stats_prep["target"]

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=100)

In [99]:
from sklearn.neighbors import KNeighborsRegressor

knn_model = KNeighborsRegressor(n_neighbors=34)

In [100]:
knn_model.fit(X_train, y_train)


KNeighborsRegressor(n_neighbors=34)

In [101]:
train_preds = knn_model.predict(X_train)

In [102]:
rmse_train = r2_score(y_train, train_preds)
rmse_train

0.386177971436666

In [103]:
test_preds = knn_model.predict(X_test)
rmse_test = r2_score(y_test, test_preds)
rmse_test

0.35015207246766

In [104]:
Y

season
2010-01-01    117.50
2010-01-01     44.12
2010-01-01    124.70
2010-01-01    218.40
2010-01-01     62.10
               ...  
2019-01-01     45.00
2019-01-01    100.90
2019-01-01    328.60
2019-01-01    278.60
2019-01-01     80.70
Name: target, Length: 2439, dtype: float64

In [105]:
combined = pd.concat([y_test, pd.Series(test_preds, index=y_test.index)], axis=1)
combined.columns = ["actual", "predictions"]

In [106]:
combined

,actual,predictions
season,,
2015-01-01,215.3,85.182353
2013-01-01,160.1,137.648824
2018-01-01,141.8,161.056471
2016-01-01,171.5,198.073529
2013-01-01,60.9,104.641176
...,...,...
2014-01-01,89.1,216.191176
2018-01-01,259.4,123.252941
2011-01-01,158.3,101.247059


In [107]:
import pickle

In [108]:
filename = "knn_model.pkl"
pickle.dump(knn_model, open(filename, 'wb'))

# 2022 Predictions

In [109]:
stats_cnx = sql.connect('../../Data/Yearly Data as Database/Stats_with_names_2021.db')
stats_2021 = pd.read_sql_query('SELECT * FROM Stats_with_names_2021 ', stats_cnx)
stats_cnx.close()

In [110]:
stats_2021['wopr'] = stats_2021['wopr_x'] + stats_2021['wopr_y']

In [111]:
stats_2021['rtd_sh'] = stats_2021['rtd_sh'].fillna(stats_2021['rtd_sh'].mean())
stats_2021['dom'] = stats_2021['dom'].fillna(stats_2021['dom'].mean())
stats_2021['w8dom'] = stats_2021['w8dom'].fillna(stats_2021['w8dom'].mean())

In [112]:
pred_test = knn_model.predict(stats_2021[predictors])

In [113]:
stats_2021['rob_fpts_2022'] = pred_test

In [114]:
stats_2021

,Name,Position,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,...,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,wopr,rob_fpts_2022
0,Tom Brady,QB,19596,2021,REG,485,719,5316.0,43,12.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,274.042941
1,Andy Lee,PN,22824,2021,REG,0,0,0.0,0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,76.015294
2,Ben Roethlisberger,QB,22924,2021,REG,390,605,3740.0,22,10.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,204.689412
3,Aaron Rodgers,QB,23459,2021,REG,366,531,4115.0,37,4.0,...,0.001939,-0.000923,0.000000,0.000000,0.000000,-0.000461,-0.000738,-0.007156,0.042417,235.290588
4,Ryan Fitzpatrick,QB,23682,2021,REG,3,6,13.0,0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,75.127059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,Hunter Long,TE,37004,2021,REG,0,0,0.0,0,0.0,...,0.047322,0.010076,0.000000,0.000000,0.000000,0.005038,0.008060,0.068966,0.197952,79.808824
640,Tommy Tremble,TE,37005,2021,REG,0,0,0.0,0,0.0,...,0.165644,0.063003,0.090909,0.074627,0.075862,0.076956,0.068584,0.360000,2.579061,92.389412
641,Tre' McKitty,TE,37011,2021,REG,0,0,0.0,0,0.0,...,0.054752,0.023974,0.000000,0.030303,0.026786,0.011987,0.019180,0.177165,0.433019,85.450000
642,Trey Lance,QB,37012,2021,REG,41,71,603.0,5,2.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,130.169412


In [115]:
cols = list(stats_2021)
cols.insert(2, cols.pop(cols.index('rob_fpts_2022')))
stats_2021 = stats_2021.loc[:, cols]
stats_2021['rob_fpts_2022'] = stats_2021['rob_fpts_2022'].apply(lambda x: round(x, 2))
stats_2021

,Name,Position,rob_fpts_2022,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,wopr
0,Tom Brady,QB,274.04,19596,2021,REG,485,719,5316.0,43,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Andy Lee,PN,76.02,22824,2021,REG,0,0,0.0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Ben Roethlisberger,QB,204.69,22924,2021,REG,390,605,3740.0,22,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Aaron Rodgers,QB,235.29,23459,2021,REG,366,531,4115.0,37,...,0.000000,0.001939,-0.000923,0.000000,0.000000,0.000000,-0.000461,-0.000738,-0.007156,0.042417
4,Ryan Fitzpatrick,QB,75.13,23682,2021,REG,3,6,13.0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,Hunter Long,TE,79.81,37004,2021,REG,0,0,0.0,0,...,0.015625,0.047322,0.010076,0.000000,0.000000,0.000000,0.005038,0.008060,0.068966,0.197952
640,Tommy Tremble,TE,92.39,37005,2021,REG,0,0,0.0,0,...,0.048985,0.165644,0.063003,0.090909,0.074627,0.075862,0.076956,0.068584,0.360000,2.579061
641,Tre' McKitty,TE,85.45,37011,2021,REG,0,0,0.0,0,...,0.036836,0.054752,0.023974,0.000000,0.030303,0.026786,0.011987,0.019180,0.177165,0.433019
642,Trey Lance,QB,130.17,37012,2021,REG,41,71,603.0,5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [116]:
predicted_fpts_2022 = stats_2021.filter(['Name','Position','rob_fpts_2022'], axis=1)

In [117]:
predicted_fpts_2022.sort_values(by=['rob_fpts_2022'], ascending=False)

,Name,Position,rob_fpts_2022
87,Derek Carr,QB,275.92
538,Justin Herbert,QB,275.39
14,Matthew Stafford,QB,274.98
0,Tom Brady,QB,274.04
255,Patrick Mahomes,QB,273.49
...,...,...,...
383,Davion Davis,WR,74.58
416,Chris Myarick,TE,74.58
491,Easop Winston,WR,74.27
480,Gabe Nabers,RB,72.81


In [118]:
top50 = predicted_fpts_2022.nlargest(50,'rob_fpts_2022')
top50['Position'].value_counts()

QB    21
WR    20
RB     6
TE     3
Name: Position, dtype: int64

In [119]:
cnx = sql.connect("Predicted_fpts_2022_KNN_40.db") #input Database Name
cursor = cnx.cursor() # Creates Cursor if needed comment out

In [120]:
predicted_fpts_2022.to_sql('Predicted_fpts_2022_KNN_15', cnx, if_exists='replace', index = False) # Exports data to an SQL Lite Database
cursor.close()
cnx.close()   

# Gridded KNN

In [206]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=100)

In [207]:
from sklearn.model_selection import GridSearchCV
parameters = {"n_neighbors": range(1, 41)}
gridsearch = GridSearchCV(KNeighborsRegressor(), parameters)
gridsearch.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': range(1, 41)})

In [208]:
gridsearch.best_params_

{'n_neighbors': 34}

In [209]:
train_preds_grid = gridsearch.predict(X_train)
train_mse = mean_squared_error(y_train, train_preds_grid)
train_rmse = np.sqrt(train_mse)
test_preds_grid = gridsearch.predict(X_test)
test_mse = mean_squared_error(y_test, test_preds_grid)
test_rmse = np.sqrt(test_mse)

In [210]:
train_rmse

63.235102360916066

In [211]:
test_rmse

64.30557401022247

In [212]:
grid_pred_test = gridsearch.predict(stats_2021[predictors])

In [213]:
stats_2021['rob_fpts_2022'] = grid_pred_test

In [214]:
cols = list(stats_2021)
cols.insert(2, cols.pop(cols.index('rob_fpts_2022')))
stats_2021 = stats_2021.loc[:, cols]
stats_2021['rob_fpts_2022'] = stats_2021['rob_fpts_2022'].apply(lambda x: round(x, 2))
stats_2021

,Name,Position,rob_fpts_2022,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,wopr
0,Tom Brady,QB,274.04,19596,2021,REG,485,719,5316.0,43,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Andy Lee,PN,76.02,22824,2021,REG,0,0,0.0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Ben Roethlisberger,QB,204.69,22924,2021,REG,390,605,3740.0,22,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Aaron Rodgers,QB,235.29,23459,2021,REG,366,531,4115.0,37,...,0.000000,0.001939,-0.000923,0.000000,0.000000,0.000000,-0.000461,-0.000738,-0.007156,0.042417
4,Ryan Fitzpatrick,QB,75.13,23682,2021,REG,3,6,13.0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,Hunter Long,TE,79.81,37004,2021,REG,0,0,0.0,0,...,0.015625,0.047322,0.010076,0.000000,0.000000,0.000000,0.005038,0.008060,0.068966,0.197952
640,Tommy Tremble,TE,92.39,37005,2021,REG,0,0,0.0,0,...,0.048985,0.165644,0.063003,0.090909,0.074627,0.075862,0.076956,0.068584,0.360000,2.579061
641,Tre' McKitty,TE,85.45,37011,2021,REG,0,0,0.0,0,...,0.036836,0.054752,0.023974,0.000000,0.030303,0.026786,0.011987,0.019180,0.177165,0.433019
642,Trey Lance,QB,130.17,37012,2021,REG,41,71,603.0,5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [215]:
predicted_fpts_2022 = stats_2021.filter(['Name','Position','rob_fpts_2022'], axis=1)

In [216]:
predicted_fpts_2022.sort_values(by=['rob_fpts_2022'], ascending=False)

,Name,Position,rob_fpts_2022
87,Derek Carr,QB,275.92
538,Justin Herbert,QB,275.39
14,Matthew Stafford,QB,274.98
0,Tom Brady,QB,274.04
255,Patrick Mahomes,QB,273.49
...,...,...,...
383,Davion Davis,WR,74.58
416,Chris Myarick,TE,74.58
491,Easop Winston,WR,74.27
480,Gabe Nabers,RB,72.81


In [217]:
top50 = predicted_fpts_2022.nlargest(50,'rob_fpts_2022')
top50['Position'].value_counts()

QB    21
WR    20
RB     6
TE     3
Name: Position, dtype: int64

In [218]:
cnx = sql.connect("Predicted_fpts_2022_KNN_Grid_Search.db") #input Database Name
cursor = cnx.cursor() # Creates Cursor if needed comment out
predicted_fpts_2022.to_sql('Predicted_fpts_2022_KNN_Grid_Search', cnx, if_exists='replace', index = False) # Exports data to an SQL Lite Database
cursor.close()
cnx.close()   